<a href="https://colab.research.google.com/github/maktaurus/ML-Work/blob/main/Custom_ResNet_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below is the custom ResNet Model

In [ ]:
class CustomConv2D(tf.keras.layers.Layer):
  def __init__(self, filters, kernal_size, n_strides, padding="valid") -> None:
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters, kernal_size, n_strides, activation="relu", padding=padding)
    self.batch_norm = tf.keras.layers.BatchNormalization()

  def call(self, x, training=True):
    x = self.conv(x)
    x = self.batch_norm(x, training)

    return x

In [ ]:
# ResidualBlock
class ResidualBlock(tf.keras.layers.Layer):
  def __init__(self, filters, n_strides=1) -> None:
    super().__init__()
    self.dotted = (n_strides !=1)
    self.custom_conv_1 = CustomConv2D(filters, 3, n_strides, padding="same")
    self.custom_conv_2 = CustomConv2D(filters, 3, 1, padding="same")
    self.activation = tf.keras.layers.Activation("relu")

    if self.dotted:
      self.custom_conv_3 = CustomConv2D(filters, 1, n_strides)

  def call(self,input,training=True):
    x = self.custom_conv_1(input, training)
    x = self.custom_conv_2(x, training)

    if self.dotted:
      x_add = self.custom_conv_3(input, training)
      x_add = tf.keras.layers.Add()([x,x_add])
    else:
      x_add = tf.keras.layers.Add()([x, input])

    return self.activation(x_add)


In [ ]:
#Resnet Model -

class ResNet34(tf.keras.Model):
  def __init__(self) -> None:
    super().__init__()

    self.conv_1 = CustomConv2D(64,7,2,padding="same")
    self.max_pool = tf.keras.layers.MaxPooling2D(3,3)

    self.conv_2_1 = ResidualBlock(64)
    self.conv_2_2 = ResidualBlock(64)
    self.conv_2_3 = ResidualBlock(64)

    self.conv_3_1 = ResidualBlock(128,2)
    self.conv_3_2 = ResidualBlock(128)
    self.conv_3_3 = ResidualBlock(128)
    self.conv_3_4 = ResidualBlock(128)

    self.conv_4_1 = ResidualBlock(256,2)
    self.conv_4_2 = ResidualBlock(256)
    self.conv_4_3 = ResidualBlock(256)
    self.conv_4_4 = ResidualBlock(256)
    self.conv_4_5 = ResidualBlock(256)
    self.conv_4_6 = ResidualBlock(256)

    self.conv_5_1 = ResidualBlock(512,2)
    self.conv_5_2 = ResidualBlock(512)
    self.conv_5_3 = ResidualBlock(512)
    self.conv_5_4 = ResidualBlock(512)

    self.global_pool = tf.keras.layers.GlobalAveragePooling2D()

    self.fc_3 = tf.keras.layers.Dense(3, activation="softmax")

  def call(self, x, training=True):
    x = self.conv_1(x)
    x = self.max_pool(x)

    x = self.conv_2_1(x, training)
    x = self.conv_2_2(x, training)
    x = self.conv_2_3(x, training)


    x = self.conv_3_1(x, training)
    x = self.conv_3_2(x, training)
    x = self.conv_3_3(x, training)
    x = self.conv_3_4(x, training)

    x = self.conv_4_1(x, training)
    x = self.conv_4_2(x, training)
    x = self.conv_4_3(x, training)
    x = self.conv_4_4(x, training)
    x = self.conv_4_5(x, training)
    x = self.conv_4_6(x, training)

    x = self.conv_5_1(x, training)
    x = self.conv_5_2(x, training)
    x = self.conv_5_3(x, training)
    x = self.conv_5_4(x, training)

    x = self.global_pool(x)

    return self.fc_3(x)

In [ ]:
res_model = ResNet34()
res_model(tf.zeros([1,255,255,3]))
res_model.summary()